# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## 問題例の読み込み
使用した問題例 : https://neo.lcc.uma.es/vrp/vrp-instances/description-for-files-of-solomons-instances/

In [ ]:
class Customer():
    def __init__(self, x, y, d, e, l, s):
        self.x = x
        self.y = y
        self.d = d
        self.e = e
        self.l = l
        self.s = s

In [ ]:
import glob

files = glob.glob("./solomon_25/*")
for file in files:
    print(file)

In [ ]:
# 問題例の選択
instance = files[0]

In [ ]:
Customers = {}
with open(instance, mode="r") as f:
    for index,line in enumerate(f):
        #print(line)
        l = []
        if index==4:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            K, Q = l
        elif 9<=index<=34:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            Customers[l[0]] = Customer(*l[1:])

In [ ]:
Customers["depot"] = Customer(0,0,0,0,1000,0)

In [ ]:
print("\t", list(vars(Customers["depot"]).keys()))
for i in Customers.keys():
    print(i, end="\t")
    for key, val in vars(Customers[i]).items():
        print(val, end=" ")
    print()

## ルートの作成

In [ ]:
# 入力
## 顧客
C={} # 客の座標を保存する辞書
TW={} # 客の時間枠を保存する辞書
demand={} # 客の要求量(正の値は集荷，負の値は配達)
S={} # 客のサービス時間

for i in Customers:
    x, y = Customers[i].x, Customers[i].y
    C[i] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[i] = (e,l)
    demand[i] = (Customers[i].d, )
    S[i] = Customers[i].s
    """name="c"+str(i)
    x, y = Customers[i].x, Customers[i].y
    C[name] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[name] = (e,l)
    demand[name] = (Customers[i].d, )
    S[name] = Customers[i].s"""
#C["depot"]=(0,0)

## 車両
M = K//2 # number of vehicles
capacities = [(Q*2, ) for k in range(M)] # capacity of vehicle

In [ ]:
print("Customer",C)
print("Time Window",TW)
print("Demand",demand)
print("Service Time",S)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(7,7))

G=nx.DiGraph()
nx.draw_networkx(G,pos=C,nodelist=[i for i in C if i != "depot"],node_color="y",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=100,with_labels=True,edge_color="k",width=1)

plt.show()

In [ ]:
# 距離関数の定義
#def Distance(t1,t2):
#    return ((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)**(0.5)
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

# ソルバーの読み込み
import sys
sys.path.append('..')

#from vrplib.vrp_d_1m1_t_model import *
import vrplib.vrp_d_1m1_t_model as vrp

# ソルバーの実行
model = vrp.Model(file.split("/")[-1].split(".")[0]) # モデルインスタンスの生成

## 客インスタンスの生成
for i in C:
    if i == "depot":
        continue
    model += vrp.Customer(i,demand=demand[i],timewindow=TW[i],servicetime=S[i])

## 車両インスタンスの生成
for k in range(M):
    model += vrp.Vehicle("v"+str(k),capacity=capacities[k])
    
## 枝インスタンスの生成
for i in C:
    for j in C:
        if i!=j:
            dist = time = distance(*C[i],*C[j])
            model += vrp.Edge(i,j,dist,time)

In [ ]:
## 最適化の実行
obj=model.optimize(IterLimit=10000,TimeLimit=10,Verbose=False,OutputFlag=False)

## 得られた解の表示
print("objective value =",obj)
for v in model.vehiclesL:
    print()
    print(v)
    print(list(map(lambda x:x.name,v.routing[1:-1])))    
    for iv in map(lambda x:x.name,v.routing[1:-1]):
        print(iv)

## ルートを1つ選ぶ

In [ ]:
# Vehicle 6の巡回路を選んでいる
tour = [12, 3, 24, 25]#+["depot"]

## その他作業

In [ ]:
# 使用した問題例の名称を保存
tmp = []
for s in instance[-1:0:-1]:
    if s=='/':
        break
    tmp.append(s)
instance_name = "".join(tmp[-1:0:-1])
print(instance_name)

# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

In [3]:
tour[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tour[-10:]

[6312, 6313, 6314, 6315, 6316, 6317, 6318, 6319, 6320, 'depot']

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [5]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [6]:
def make_preceding_constr(tour, Customers, Ax, Ap, b, c):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            i_next = "depot"
            continue
        Ax.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ap.append([0 for _ in range(len(tour))])
        b.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
    return Ax, Ap, b, c

In [7]:
def make_tw_constr(tour, Customers, Ax, Ap, b, c):
    # 時間枠制約
    for index, i in enumerate(tour):
        Ax.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(-Customers[i].e)
        Ax.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(Customers[i].l)
    return Ax, Ap, b, c

In [8]:
def make_inputs(tour, Customers):
    import numpy as np
    Ax, Ap, b, c = [], [], [], [1 for _ in range(len(tour))]
    # ルート内の顧客の順序に関する制約
    Ax, Ap, b, c = make_preceding_constr(tour, Customers, Ax, Ap, b, c)
    print("Preceding constraints are done.")
    # 時間枠制約
    Ax, Ap, b, c = make_tw_constr(tour, Customers, Ax, Ap, b, c)
    print("Time-window constarints are done.")
    # ndarrayに変換
    Ax = np.array(Ax)
    Ap = np.array(Ap)
    b = np.array(b)
    c = np.array(c)
    return Ax, Ap, b, c

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [9]:
def solve_primal(Ax, Ap, b, c, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    x_i : 顧客iへ車両が到着する時刻を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    """
    x = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="x")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c.T @ p, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax @ x + Ap @ p <= b, name="c")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## 双対問題を解く関数の定義

In [19]:
def solve_dual(Ax, Ap, b, c, instance_name, num_vars, basis_like):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(b.T @ y, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax.T @ y >= 0, name="c1")
    m.addConstr(Ap.T @ y + c >= 0, name="c2")
    m.addConstr(y >= 0, name="c3")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i in range(num_vars):
        y[i].PStart = basis_like[i]
    
    # 最適化
    #m.setParam("Method", 0)
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [11]:
Ax, Ap, b, c = make_inputs(tour, Customers)

Preceding constraints are done.
Time-window constarints are done.


In [12]:
# Gurobiによって最適解を求める
whole_x = solve_primal(Ax, Ap, b, c, instance_name+"whole_x", len(tour))

Using license file /Users/okamoto/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 18965 rows, 12644 columns and 37930 nonzeros
Model fingerprint: 0xa123105c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 12583 rows and 6264 columns
Presolve time: 0.21s
Presolved: 6382 rows, 6380 columns, 12764 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 32
 AA' NZ     : 1.527e+04
 Factor NZ  : 2.243e+04 (roughly 5 MBytes of memory)
 Factor Ops : 2.991e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.50373673e+08 -1.00000000e+03  1.06e+04 0.00e+00  1.

車両が顧客133に到着する時刻は、[835.1100264]
車両が顧客134に到着する時刻は、[838.71557768]
車両が顧客135に到着する時刻は、[841.87785534]
車両が顧客136に到着する時刻は、[849.15796523]
車両が顧客137に到着する時刻は、[853.15796523]
車両が顧客138に到着する時刻は、[860.77373833]
車両が顧客139に到着する時刻は、[862.77373833]
車両が顧客140に到着する時刻は、[868.85650086]
車両が顧客141に到着する時刻は、[875.18105619]
車両が顧客142に到着する時刻は、[877.41712416]
車両が顧客143に到着する時刻は、[878.41712416]
車両が顧客144に到着する時刻は、[882.88926012]
車両が顧客145に到着する時刻は、[887.88926012]
車両が顧客146に到着する時刻は、[892.1319008]
車両が顧客147に到着する時刻は、[894.36796878]
車両が顧客148に到着する時刻は、[903.80194991]
車両が顧客149に到着する時刻は、[909.45880416]
車両が顧客150に到着する時刻は、[911.45880416]
車両が顧客151に到着する時刻は、[917.54156669]
車両が顧客152に到着する時刻は、[919.77763467]
車両が顧客153に到着する時刻は、[930.54796429]
車両が顧客154に到着する時刻は、[935.93312909]
車両が顧客155に到着する時刻は、[939.53868037]
車両が顧客156に到着する時刻は、[942.53868037]
車両が顧客157に到着する時刻は、[947.53868037]
車両が顧客158に到着する時刻は、[954.74978292]
車両が顧客159に到着する時刻は、[962.5600326]
車両が顧客160に到着する時刻は、[972.60990822]
車両が顧客161に到着する時刻は、[976.21545949]
車両が顧客162に到着する時刻は、[981.6006243]
車両が顧客163に到着する時刻は、[985.84326499]
車両が顧客164に到着す

車両が顧客395に到着する時刻は、[2328.59199078]
車両が顧客396に到着する時刻は、[2335.30019471]
車両が顧客397に到着する時刻は、[2343.30019471]
車両が顧客398に到着する時刻は、[2348.95704896]
車両が顧客399に到着する時刻は、[2353.42918492]
車両が顧客400に到着する時刻は、[2364.05933073]
車両が顧客401に到着する時刻は、[2371.33944062]
車両が顧客402に到着する時刻は、[2381.53747964]
車両が顧客403に到着する時刻は、[2386.63649916]
車両が顧客404に到着する時刻は、[2394.25227226]
車両が顧客405に到着する時刻は、[2397.25227226]
車両が顧客406に到着する時刻は、[2400.08069939]
車両が顧客407に到着する時刻は、[2405.4658642]
車両が顧客408に到着する時刻は、[2415.4658642]
車両が顧客409に到着する時刻は、[2424.95269718]
車両が顧客410に到着する時刻は、[2433.19890843]
車両が顧客411に到着する時刻は、[2440.26997624]
車両が顧客412に到着する時刻は、[2442.26997624]
車両が顧客413に到着する時刻は、[2453.93188003]
車両が顧客414に到着する時刻は、[2463.36586116]
車両が顧客415に到着する時刻は、[2472.4212463]
車両が顧客416に到着する時刻は、[2476.54435193]
車両が顧客417に到着する時刻は、[2481.92951673]
車両が顧客418に到着する時刻は、[2491.92951673]
車両が顧客419に到着する時刻は、[2494.92951673]
車両が顧客420に到着する時刻は、[2495.92951673]
車両が顧客421に到着する時刻は、[2500.92951673]
車両が顧客422に到着する時刻は、[2510.14906119]
車両が顧客423に到着する時刻は、[2513.75461247]
車両が顧客424に到着する時刻は、[2523.24144545]
車両が顧客425に到着する

車両が顧客645に到着する時刻は、[3691.12753705]
車両が顧客646に到着する時刻は、[3695.25064268]
車両が顧客647に到着する時刻は、[3698.41292034]
車両が顧客648に到着する時刻は、[3705.48398815]
車両が顧客649に到着する時刻は、[3710.58300766]
車両が顧客650に到着する時刻は、[3717.79411022]
車両が顧客651に到着する時刻は、[3729.45601401]
車両が顧客652に到着する時刻は、[3733.57911963]
車両が顧客653に到着する時刻は、[3740.65018744]
車両が顧客654に到着する時刻は、[3747.72125525]
車両が顧客655に到着する時刻は、[3749.95732323]
車両が顧客656に到着する時刻は、[3754.42945919]
車両が顧客657に到着する時刻は、[3758.03501046]
車両が顧客658に到着する時刻は、[3763.03501046]
車両が顧客659に到着する時刻は、[3765.27107844]
車両が顧客660に到着する時刻は、[3771.59563376]
車両が顧客661に到着する時刻は、[3778.30383769]
車両が顧客662に到着する時刻は、[3780.53990567]
車両が顧客663に到着する時刻は、[3783.70218333]
車両が顧客664に到着する時刻は、[3785.11639689]
車両が顧客665に到着する時刻は、[3788.27867455]
車両が顧客666に到着する時刻は、[3791.44095221]
車両が顧客667に到着する時刻は、[3795.56405784]
車両が顧客668に到着する時刻は、[3801.64682037]
車両が顧客669に到着する時刻は、[3802.64682037]
車両が顧客670に到着する時刻は、[3808.97137569]
車両が顧客671に到着する時刻は、[3811.79980281]
車両が顧客672に到着する時刻は、[3819.86206056]
車両が顧客673に到着する時刻は、[3821.27627412]
車両が顧客674に到着する時刻は、[3824.43855178]
車両が顧客675に到

車両が顧客896に到着する時刻は、[5114.45615033]
車両が顧客897に到着する時刻は、[5118.45615033]
車両が顧客898に到着する時刻は、[5128.50602595]
車両が顧客899に到着する時刻は、[5135.7171285]
車両が顧客900に到着する時刻は、[5138.87940616]
車両が顧客901に到着する時刻は、[5141.11547414]
車両が顧客902に到着する時刻は、[5149.36168539]
車両が顧客903に到着する時刻は、[5158.3059573]
車両が顧客904に到着する時刻は、[5162.42906292]
車両が顧客905に到着する時刻は、[5168.51182545]
車両が顧客906に到着する時刻は、[5175.58289327]
車両が顧客907に到着する時刻は、[5179.58289327]
車両が顧客908に到着する時刻は、[5184.96805807]
車両が顧客909に到着する時刻は、[5186.38227164]
車両が顧客910に到着する時刻は、[5190.38227164]
車両が顧客911に到着する時刻は、[5200.23112944]
車両が顧客912に到着する時刻は、[5202.23112944]
車両が顧客913に到着する時刻は、[5212.07998724]
車両が顧客914に到着する時刻は、[5216.20309286]
車両が顧客915に到着する時刻は、[5221.20309286]
車両が顧客916に到着する時刻は、[5226.30211238]
車両が顧客917に到着する時刻は、[5236.15097018]
車両が顧客918に到着する時刻は、[5243.22203799]
車両が顧客919に到着する時刻は、[5249.05298989]
車両が顧客920に到着する時刻は、[5250.05298989]
車両が顧客921に到着する時刻は、[5258.11524763]
車両が顧客922に到着する時刻は、[5264.82345157]
車両が顧客923に到着する時刻は、[5267.05951954]
車両が顧客924に到着する時刻は、[5270.05951954]
車両が顧客925に到着する時刻は、[5273.66507082]
車両が顧客926に到着す

...

車両が顧客5112に到着する時刻は、[29038.00813107]
車両が顧客5113に到着する時刻は、[29049.67003486]
車両が顧客5114に到着する時刻は、[29059.10401599]
車両が顧客5115に到着する時刻は、[29059.10401599]
車両が顧客5116に到着する時刻は、[29063.22712162]
車両が顧客5117に到着する時刻は、[29069.05807351]
車両が顧客5118に到着する時刻は、[29074.15709303]
車両が顧客5119に到着する時刻は、[29079.25611254]
車両が顧客5120に到着する時刻は、[29087.74139392]
車両が顧客5121に到着する時刻は、[29093.74139392]
車両が顧客5122に到着する時刻は、[29096.90367158]
車両が顧客5123に到着する時刻は、[29104.90367158]
車両が顧客5124に到着する時刻は、[29116.30542583]
車両が顧客5125に到着する時刻は、[29121.40444534]
車両が顧客5126に到着する時刻は、[29126.50346485]
車両が顧客5127に到着する時刻は、[29133.71456741]
車両が顧客5128に到着する時刻は、[29139.09973221]
車両が顧客5129に到着する時刻は、[29151.14132679]
車両が顧客5130に到着する時刻は、[29162.54308104]
車両が顧客5131に到着する時刻は、[29169.82319093]
車両が顧客5132に到着する時刻は、[29176.14774625]
車両が顧客5133に到着する時刻は、[29178.38381423]
車両が顧客5134に到着する時刻は、[29183.48283374]
車両が顧客5135に到着する時刻は、[29188.48283374]
車両が顧客5136に到着する時刻は、[29197.7023782]
車両が顧客5137に到着する時刻は、[29197.7023782]
車両が顧客5138に到着する時刻は、[29206.24638195]
車両が顧客5139に到着する時刻は、[29212.32914448]
車両が顧客5140に到着する時刻は、[292

車両が顧客5360に到着する時刻は、[30567.97177544]
車両が顧客5361に到着する時刻は、[30573.07079496]
車両が顧客5362に到着する時刻は、[30576.67634623]
車両が顧客5363に到着する時刻は、[30580.28189751]
車両が顧客5364に到着する時刻は、[30588.52810876]
車両が顧客5365に到着する時刻は、[30598.52810876]
車両が顧客5366に到着する時刻は、[30603.52810876]
車両が顧客5367に到着する時刻は、[30610.59917657]
車両が顧客5368に到着する時刻は、[30620.59917657]
車両が顧客5369に到着する時刻は、[30625.07131253]
車両が顧客5370に到着する時刻は、[30628.6768638]
車両が顧客5371に到着する時刻は、[30629.6768638]
車両が顧客5372に到着する時刻は、[30638.62113571]
車両が顧客5373に到着する時刻は、[30645.69220352]
車両が顧客5374に到着する時刻は、[30648.85448118]
車両が顧客5375に到着する時刻は、[30656.85448118]
車両が顧客5376に到着する時刻は、[30663.56268512]
車両が顧客5377に到着する時刻は、[30667.56268512]
車両が顧客5378に到着する時刻は、[30670.72496278]
車両が顧客5379に到着する時刻は、[30680.62445771]
車両が顧客5380に到着する時刻は、[30691.25460353]
車両が顧客5381に到着する時刻は、[30692.66881709]
車両が顧客5382に到着する時刻は、[30699.07194133]
車両が顧客5383に到着する時刻は、[30700.48615489]
車両が顧客5384に到着する時刻は、[30705.48615489]
車両が顧客5385に到着する時刻は、[30713.73236614]
車両が顧客5386に到着する時刻は、[30715.1465797]
車両が顧客5387に到着する時刻は、[30716.56079326]
車両が顧客5388に到着する時刻は、[3072

車両が顧客5639に到着する時刻は、[32167.11056169]
車両が顧客5640に到着する時刻は、[32172.11056169]
車両が顧客5641に到着する時刻は、[32177.94151358]
車両が顧客5642に到着する時刻は、[32180.76994071]
車両が顧客5643に到着する時刻は、[32187.84100852]
車両が顧客5644に到着する時刻は、[32195.84100852]
車両が顧客5645に到着する時刻は、[32204.89639366]
車両が顧客5646に到着する時刻は、[32205.89639366]
車両が顧客5647に到着する時刻は、[32211.97915619]
車両が顧客5648に到着する時刻は、[32216.97915619]
車両が顧客5649に到着する時刻は、[32222.81010808]
車両が顧客5650に到着する時刻は、[32229.81010808]
車両が顧客5651に到着する時刻は、[32237.87236583]
車両が顧客5652に到着する時刻は、[32244.27549007]
車両が顧客5653に到着する時刻は、[32246.51155805]
車両が顧客5654に到着する時刻は、[32256.95186456]
車両が顧客5655に到着する時刻は、[32260.11414222]
車両が顧客5656に到着する時刻は、[32263.27641988]
車両が顧客5657に到着する時刻は、[32273.27641988]
車両が顧客5658に到着する時刻は、[32277.27641988]
車両が顧客5659に到着する時刻は、[32279.27641988]
車両が顧客5660に到着する時刻は、[32287.82042362]
車両が顧客5661に到着する時刻は、[32291.82042362]
車両が顧客5662に到着する時刻は、[32292.82042362]
車両が顧客5663に到着する時刻は、[32300.03152617]
車両が顧客5664に到着する時刻は、[32307.31163606]
車両が顧客5665に到着する時刻は、[32314.59174595]
車両が顧客5666に到着する時刻は、[32318.71485158]
車両が顧客5667に到着する時刻は、[3

車両が顧客5873に到着する時刻は、[33486.53296849]
車両が顧客5874に到着する時刻は、[33490.65607411]
車両が顧客5875に到着する時刻は、[33496.04123892]
車両が顧客5876に到着する時刻は、[33505.04123892]
車両が顧客5877に到着する時刻は、[33509.04123892]
車両が顧客5878に到着する時刻は、[33513.51337488]
車両が顧客5879に到着する時刻は、[33516.51337488]
車両が顧客5880に到着する時刻は、[33528.71993049]
車両が顧客5881に到着する時刻は、[33534.80269302]
車両が顧客5882に到着する時刻は、[33536.21690659]
車両が顧客5883に到着する時刻は、[33539.37918425]
車両が顧客5884に到着する時刻は、[33542.54146191]
車両が顧客5885に到着する時刻は、[33546.78410259]
車両が顧客5886に到着する時刻は、[33550.90720822]
車両が顧客5887に到着する時刻は、[33556.98997075]
車両が顧客5888に到着する時刻は、[33560.98997075]
車両が顧客5889に到着する時刻は、[33562.98997075]
車両が顧客5890に到着する時刻は、[33566.59552202]
車両が顧客5891に到着する時刻は、[33569.59552202]
車両が顧客5892に到着する時刻は、[33572.59552202]
車両が顧客5893に到着する時刻は、[33581.53979393]
車両が顧客5894に到着する時刻は、[33585.53979393]
車両が顧客5895に到着する時刻は、[33587.77586191]
車両が顧客5896に到着する時刻は、[33596.72013382]
車両が顧客5897に到着する時刻は、[33602.80289635]
車両が顧客5898に到着する時刻は、[33609.12745167]
車両が顧客5899に到着する時刻は、[33618.61428465]
車両が顧客5900に到着する時刻は、[33621.44271178]
車両が顧客5901に到着する時刻は、[3

車両が顧客6198に到着する時刻は、[35277.28959759]
車両が顧客6199に到着する時刻は、[35281.76173354]
車両が顧客6200に到着する時刻は、[35283.99780152]
車両が顧客6201に到着する時刻は、[35288.46993748]
車両が顧客6202に到着する時刻は、[35292.07548875]
車両が顧客6203に到着する時刻は、[35294.31155673]
車両が顧客6204に到着する時刻は、[35296.54762471]
車両が顧客6205に到着する時刻は、[35301.54762471]
車両が顧客6206に到着する時刻は、[35306.01976066]
車両が顧客6207に到着する時刻は、[35308.25582864]
車両が顧客6208に到着する時刻は、[35313.91268289]
車両が顧客6209に到着する時刻は、[35321.52845599]
車両が顧客6210に到着する時刻は、[35330.74800045]
車両が顧客6211に到着する時刻は、[35336.83076298]
車両が顧客6212に到着する時刻は、[35340.83076298]
車両が顧客6213に到着する時刻は、[35347.53896691]
車両が顧客6214に到着する時刻は、[35360.99259096]
車両が顧客6215に到着する時刻は、[35365.11569659]
車両が顧客6216に到着する時刻は、[35374.11569659]
車両が顧客6217に到着する時刻は、[35383.54967772]
車両が顧客6218に到着する時刻は、[35388.54967772]
車両が顧客6219に到着する時刻は、[35393.93484253]
車両が顧客6220に到着する時刻は、[35396.1709105]
車両が顧客6221に到着する時刻は、[35401.26993002]
車両が顧客6222に到着する時刻は、[35405.26993002]
車両が顧客6223に到着する時刻は、[35410.36894953]
車両が顧客6224に到着する時刻は、[35416.19990143]
車両が顧客6225に到着する時刻は、[35421.58506623]
車両が顧客6226に到着する時刻は、[35

In [13]:
for var in whole_x.getVars():
    print(var.varName, var.X)

x[0] 0.0
x[1] 6.082762530298219
x[2] 9.688313805762208
x[3] 15.519265700607509
x[4] 22.51926570060751
x[5] 24.51926570060751
x[6] 35.83297419959227
x[7] 47.23472845058365
x[8] 55.778732195901185
x[9] 59.38428347136517
x[10] 67.44654121966371
x[11] 74.15474515216309
x[12] 84.97139897855506
x[13] 90.80235087340036
x[14] 94.80235087340036
x[15] 97.03841885090014
x[16] 99.27448682839993
x[17] 105.27448682839993
x[18] 108.27448682839993
x[19] 112.74662278339952
x[20] 114.16083634577261
x[21] 116.16083634577261
x[22] 122.48539166610936
x[23] 126.60849729172702
x[24] 133.88860718100753
x[25] 141.69885685691418
x[26] 152.51551068330613
x[27] 160.57776843160468
x[28] 168.38801810751133
x[29] 176.8732994817499
x[30] 182.95606201204814
x[31] 193.25169215303515
x[32] 200.46279470396314
x[33] 207.67389725489113
x[34] 209.90996523239093
x[35] 211.32417879476404
x[36] 216.42319830835683
x[37] 219.5854759685252
x[38] 229.8811061095122
x[39] 241.19481460849696
x[40] 253.23640918728924
x[41] 262.1806810

x[357] 2096.019653707366
x[358] 2103.63542681323
x[359] 2107.63542681323
x[360] 2114.7064946250953
x[361] 2121.986604514376
x[362] 2125.1488821745443
x[363] 2130.1488821745443
x[364] 2138.1488821745443
x[365] 2146.211139922843
x[366] 2151.8679941723353
x[367] 2158.1925494926722
x[368] 2162.6646854476717
x[369] 2165.82696310784
x[370] 2168.9892407680086
x[371] 2177.933512678008
x[372] 2184.336636915441
x[373] 2186.5727048929407
x[374] 2194.3829545688473
x[375] 2204.432830189968
x[376] 2207.432830189968
x[377] 2216.0351554570107
x[378] 2220.1582610826285
x[379] 2227.229328894494
x[380] 2229.4653968719936
x[381] 2236.6764994229216
x[382] 2245.2205031682392
x[383] 2249.6926391232387
x[384] 2254.7916586368315
x[385] 2257.027726614331
x[386] 2263.1104891446294
x[387] 2268.767343394122
x[388] 2278.8172190152427
x[389] 2290.218973266234
x[390] 2296.0499251610795
x[391] 2300.2925658481986
x[392] 2306.3753283784968
x[393] 2314.4375861267954
x[394] 2323.4929712649327
x[395] 2328.5919907785255
x[3

x[874] 4963.949613224229
x[875] 4972.493616969547
x[876] 4979.564684781412
x[877] 4980.564684781412
x[878] 4986.967809018845
x[879] 4992.3529738259795
x[880] 5000.415231574278
x[881] 5004.538337199895
x[882] 5010.862892520232
x[883] 5020.082436977525
x[884] 5027.15350478939
x[885] 5036.208889927528
x[886] 5039.208889927528
x[887] 5042.8144412029915
x[888] 5052.248422335048
x[889] 5062.878568147783
x[890] 5065.114636125283
x[891] 5075.114636125283
x[892] 5085.744781938018
x[893] 5093.807039686316
x[894] 5096.969317346484
x[895] 5104.969317346484
x[896] 5114.456150326989
x[897] 5118.456150326989
x[898] 5128.50602594811
x[899] 5135.717128499037
x[900] 5138.879406159205
x[901] 5141.115474136705
x[902] 5149.361685387941
x[903] 5158.30595729794
x[904] 5162.4290629235575
x[905] 5168.511825453856
x[906] 5175.582893265721
x[907] 5179.582893265721
x[908] 5184.9680580728555
x[909] 5186.382271635229
x[910] 5190.382271635229
x[911] 5200.231129437025
x[912] 5202.231129437025
x[913] 5212.079987238821

x[1373] 7777.451937960618
x[1374] 7783.2828898554635
x[1375] 7787.405995481081
x[1376] 7795.2162451569875
x[1377] 7800.2162451569875
x[1378] 7805.601409964122
x[1379] 7810.986574771256
x[1380] 7817.3111300915925
x[1381] 7820.3111300915925
x[1382] 7830.941275904327
x[1383] 7832.941275904327
x[1384] 7833.941275904327
x[1385] 7841.557049010191
x[1386] 7847.960173247624
x[1387] 7852.960173247624
x[1388] 7857.960173247624
x[1389] 7864.284728567961
x[1390] 7864.284728567961
x[1391] 7875.05505818223
x[1392] 7884.2746026395225
x[1393] 7893.2746026395225
x[1394] 7902.2746026395225
x[1395] 7909.2746026395225
x[1396] 7914.931456889015
x[1397] 7917.167524866515
x[1398] 7921.290630492133
x[1399] 7925.762766447132
x[1400] 7932.833834258998
x[1401] 7942.682692060794
x[1402] 7950.928903312029
x[1403] 7953.16497128953
x[1404] 7954.16497128953
x[1405] 7961.236039101395
x[1406] 7969.2982968496935
x[1407] 7972.903848125157
x[1408] 7975.1399161026575
x[1409] 7976.554129665031
x[1410] 7977.554129665031
x[14

x[1872] 10676.187196374349
x[1873] 10683.802969480214
x[1874] 10684.802969480214
x[1875] 10687.965247140382
x[1876] 10693.965247140382
x[1877] 10700.048009670681
x[1878] 10704.520145625682
x[1879] 10711.520145625682
x[1880] 10721.419640562293
x[1881] 10728.490708374158
x[1882] 10735.490708374158
x[1883] 10738.652986034327
x[1884] 10740.652986034327
x[1885] 10744.776091659945
x[1886] 10750.432945909437
x[1887] 10753.261373034184
x[1888] 10756.261373034184
x[1889] 10762.092324929028
x[1890] 10769.092324929028
x[1891] 10775.495449166461
x[1892] 10780.880613973595
x[1893] 10789.824885883594
x[1894] 10791.239099445967
x[1895] 10799.301357194267
x[1896] 10809.200852130878
x[1897] 10812.363129791047
x[1898] 10817.46214930464
x[1899] 10826.896130436697
x[1900] 10832.896130436697
x[1901] 10840.958388184996
x[1902] 10844.120665845165
x[1903] 10855.43437434415
x[1904] 10861.091228593641
x[1905] 10866.476393400775
x[1906] 10873.756503290055
x[1907] 10879.141668097189
x[1908] 10883.264773722807
x[1

...

p[4290] 9386.222730656096
p[4291] 9385.222730656096
p[4292] 9383.694866611095
p[4293] 9381.109080173468
p[4294] 9383.494244980604
p[4295] 9382.577007510903
p[4296] 9380.700113136521
p[4297] 9384.771180948388
p[4298] 9390.771180948388
p[4299] 9386.933458608557
p[4300] 9384.761885733304
p[4301] 9384.924163393473
p[4302] 9391.006925923772
p[4303] 9395.006925923772
p[4304] 9398.55092966909
p[4305] 9399.71320732926
p[4306] 9399.875484989429
p[4307] 9401.260649796564
p[4308] 9400.089076921311
p[4309] 9408.287115948497
p[4310] 9413.349373696794
p[4311] 9414.673929017132
p[4312] 9414.673929017132
p[4313] 9411.909996994633
p[4314] 9413.324210557006
p[4315] 9413.73842411938
p[4316] 9415.018534008661
p[4317] 9415.624085284126
p[4318] 9415.860153261627
p[4319] 9422.676807088017
p[4320] 9431.338710877706
p[4321] 9437.187568679503
p[4322] 9435.187568679503
p[4323] 9435.187568679503
p[4324] 9432.423636657004
p[4325] 9436.506399187303
p[4326] 9441.122172293166
p[4327] 9436.53638585554
p[4328] 9440.659

p[4729] 10364.566698065857
p[4730] 10366.172249341322
p[4731] 10370.829103590815
p[4732] 10369.065171568316
p[4733] 10370.307812255436
p[4734] 10369.543880232937
p[4735] 10370.706157893106
p[4736] 10376.706157893106
p[4737] 10376.534585017853
p[4738] 10381.150358123716
p[4739] 10385.205743261853
p[4740] 10390.403782289039
p[4741] 10386.232209413785
p[4742] 10386.060636538532
p[4743] 10389.891588433376
p[4744] 10390.363724388375
p[4745] 10388.486830013993
p[4746] 10393.486830013993
p[4747] 10394.609935639612
p[4748] 10394.708955153204
p[4749] 10391.31450642867
p[4750] 10396.858510173988
p[4751] 10395.272723736362
p[4752] 10396.272723736362
p[4753] 10396.371743249954
p[4754] 10400.812049758864
p[4755] 10407.883117570731
p[4756] 10410.485442837773
p[4757] 10411.608548463391
p[4758] 10409.844616440892
p[4759] 10410.229781248028
p[4760] 10409.46584922553
p[4761] 10407.588954851148
p[4762] 10410.712060476766
p[4763] 10410.712060476766
p[4764] 10418.560918278563
p[4765] 10419.8071295298
p[476

p[5289] 11682.227752354178
p[5290] 11683.056179478925
p[5291] 11682.056179478925
p[5292] 11683.528315433923
p[5293] 11687.627334947516
p[5294] 11689.627334947516
p[5295] 11694.907444836797
p[5296] 11696.232000157135
p[5297] 11696.474640844255
p[5298] 11701.530025982393
p[5299] 11705.530025982393
p[5300] 11709.772666669513
p[5301] 11716.582916345418
p[5302] 11718.681935859011
p[5303] 11725.492185534917
p[5304] 11731.926166666974
p[5305] 11737.696496281242
p[5306] 11741.696496281242
p[5307] 11742.77925881154
p[5308] 11748.628116613338
p[5309] 11749.336320545837
p[5310] 11747.808456500836
p[5311] 11748.133011821174
p[5312] 11753.133011821174
p[5313] 11747.133011821174
p[5314] 11752.963963716018
p[5315] 11755.087069341636
p[5316] 11759.7028424475
p[5317] 11764.913944998429
p[5318] 11764.037050624047
p[5319] 11767.092435762184
p[5320] 11774.722581574919
p[5321] 11772.194717529917
p[5322] 11778.256975278215
p[5323] 11783.537085167496
p[5324] 11791.353738993887
p[5325] 11798.57328345118
p[532

p[5696] 12601.831881303693
p[5697] 12596.831881303693
p[5698] 12597.447654409556
p[5699] 12598.683722387057
p[5700] 12596.919790364558
p[5701] 12602.321544615548
p[5702] 12606.519583642734
p[5703] 12613.519583642734
p[5704] 12618.063587388053
p[5705] 12620.225865048222
p[5706] 12622.054292172968
p[5707] 12621.29036015047
p[5708] 12622.413465776088
p[5709] 12625.019017051553
p[5710] 12621.019017051553
p[5711] 12622.727220984052
p[5712] 12629.435424916552
p[5713] 12634.75998023689
p[5714] 12641.822237985187
p[5715] 12651.671095786984
p[5716] 12653.995651107321
p[5717] 12650.409864669695
p[5718] 12651.025637775558
p[5719] 12657.087895523855
p[5720] 12655.560031478853
p[5721] 12661.37028115476
p[5722] 12662.432538903056
p[5723] 12662.555644528675
p[5724] 12663.212498778168
p[5725] 12660.212498778168
p[5726] 12658.212498778168
p[5727] 12666.614253029158
p[5728] 12667.860464280395
p[5729] 12668.860464280395
p[5730] 12673.943226810694
p[5731] 12674.32839161783
p[5732] 12677.390649366127
p[573

p[6172] 13582.962082759164
p[6173] 13586.618937008658
p[6174] 13587.224488284119
p[6175] 13593.444032741412
p[6176] 13593.049584016873
p[6177] 13596.99385592687
p[6178] 13601.938127836867
p[6179] 13607.218237726149
p[6180] 13609.27362286429
p[6181] 13607.102049989036
p[6182] 13603.102049989036
p[6183] 13604.810253921532
p[6184] 13604.810253921532
p[6185] 13607.972531581698
p[6186] 13608.800958706444
p[6187] 13617.020503163738
p[6188] 13617.434716726108
p[6189] 13620.378988636105
p[6190] 13625.428864257228
p[6191] 13622.428864257228
p[6192] 13632.245518083619
p[6193] 13638.679499215672
p[6194] 13640.387703148168
p[6195] 13642.549980808333
p[6196] 13642.549980808333
p[6197] 13644.632743338632
p[6198] 13647.289597588126
p[6199] 13651.761733543128
p[6200] 13649.997801520629
p[6201] 13649.469937475631
p[6202] 13652.075488751092
p[6203] 13652.311556728593
p[6204] 13650.547624706094
p[6205] 13652.547624706094
p[6206] 13652.019760661096
p[6207] 13651.255828638597
p[6208] 13654.91268288809
p[62

In [ ]:
for constr in whole_x.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [14]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
#print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

前半は、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

### 2. 前半後半それぞれのPrimalを解く

In [15]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for Ax_i, Ap_i, b_i in zip(Ax, Ap, b):
    if np.linalg.norm(Ax_i[threshold:], ord=2)==0.:
        A1x.append(Ax_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
    elif np.linalg.norm(Ax_i[:threshold], ord=2)==0.:
        A2x.append(Ax_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
    else:
        A3x.append(Ax_i)
        A3p.append(Ap_i)
        b3.append(b_i)
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

A1x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A1p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b1 : [-6.08276253e+00 -3.60555128e+00 -5.83095189e+00 ...  1.10180000e+04
 -1.10190000e+04  1.10230000e+04]
[1 1 1 ... 1 1 1]
A2x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A2p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b2 : [-1.00000000e+01 -6.70820393e+00 -8.06225775e+00 ...  2.20510000e+04
  0.00000000e+00  1.00000000e+03]
[1 1 1 ... 1 1 1]
A3x : [[0 0 0 ... 0 0 0]]
A3p : [[0 0 0 ... 0 0 0]]
b3 : [-8.06225775]


In [16]:
# Gurobiによって最適解を求める
former_x = solve_primal(A1x, A1p, b1, c1, instance_name+"former_x", len(former))
latter_x = solve_primal(A2x, A2p, b2, c2, instance_name+"latter_x", len(latter))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 9482 rows, 6322 columns and 18964 nonzeros
Model fingerprint: 0x79f6c9b9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 6294 rows and 3136 columns
Presolve time: 0.08s
Presolved: 3188 rows, 3186 columns, 6376 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0943210e+07   4.794063e+00   0.000000e+00      0s
    2980    1.0954124e+07   0.000000e+00   0.000000e+00      0s

Solved in 2980 iterations and 0.29 seconds
Optimal objective  1.095412389e+07
車両が顧客0に到着する時刻は、[0.]
車両が顧客1に到着する時刻は、[6.08276253]
車両が顧客2に到着する時刻は、[9.68831381]
車両が顧客3に到着する時刻は、[15.5192657]
車両が顧客4に到着する時刻は、[22.5192657]
車両が顧客5に到着する時刻は、[24.5192657]
車両が顧客6に到着する時刻は、[35.8329742]
車両が顧客7に到着する時刻は、[47.23472845]
車両が顧客8に到着する時刻は、[55.7787322]
車両が顧客9に到着する時刻は、[59.38428347]
車両が顧客10に到着する時刻は、[67.44654122]

車両が顧客257に到着する時刻は、[1516.74166623]
車両が顧客258に到着する時刻は、[1523.74166623]
車両が顧客259に到着する時刻は、[1529.12683104]
車両が顧客260に到着する時刻は、[1538.56081217]
車両が顧客261に到着する時刻は、[1549.33114179]
車両が顧客262に到着する時刻は、[1551.33114179]
車両が顧客263に到着する時刻は、[1555.57378247]
車両が顧客264に到着する時刻は、[1562.78488503]
車両が顧客265に到着する時刻は、[1568.86764756]
車両が顧客266に到着する時刻は、[1576.9299053]
車両が顧客267に到着する時刻は、[1585.53223057]
車両が顧客268に到着する時刻は、[1587.53223057]
車両が顧客269に到着する時刻は、[1592.63125008]
車両が顧客270に到着する時刻は、[1596.63125008]
車両が顧客271に到着する時刻は、[1601.63125008]
車両が顧客272に到着する時刻は、[1603.63125008]
車両が顧客273に到着する時刻は、[1609.63125008]
車両が顧客274に到着する時刻は、[1616.63125008]
車両が顧客275に到着する時刻は、[1623.84235264]
車両が顧客276に到着する時刻は、[1628.84235264]
車両が顧客277に到着する時刻は、[1634.67330453]
車両が顧客278に到着する時刻は、[1638.79641016]
車両が顧客279に到着する時刻は、[1641.79641016]
車両が顧客280に到着する時刻は、[1646.26854611]
車両が顧客281に到着する時刻は、[1649.43082377]
車両が顧客282に到着する時刻は、[1651.43082377]
車両が顧客283に到着する時刻は、[1654.59310143]
車両が顧客284に到着する時刻は、[1660.42405333]
車両が顧客285に到着する時刻は、[1665.80921813]
車両が顧客286に到着する時刻は、[1669.41476941]
車両が顧客287に到着

車両が顧客794に到着する時刻は、[4478.19570452]
車両が顧客795に到着する時刻は、[4482.31881014]
車両が顧客796に到着する時刻は、[4485.92436142]
車両が顧客797に到着する時刻は、[4488.1604294]
車両が顧客798に到着する時刻は、[4495.97067907]
車両が顧客799に到着する時刻は、[4505.19022353]
車両が顧客800に到着する時刻は、[4513.43643478]
車両が顧客801に到着する時刻は、[4519.7609901]
車両が顧客802に到着する時刻は、[4525.84375263]
車両が顧客803に到着する時刻は、[4528.07982061]
車両が顧客804に到着する時刻は、[4533.9107725]
車両が顧客805に到着する時刻は、[4535.9107725]
車両が顧客806に到着する時刻は、[4543.12187506]
車両が顧客807に到着する時刻は、[4543.12187506]
車両が顧客808に到着する時刻は、[4551.12187506]
車両が顧客809に到着する時刻は、[4559.1841328]
車両が顧客810に到着する時刻は、[4568.23951794]
車両が顧客811に到着する時刻は、[4572.36262357]
車両が顧客812に到着する時刻は、[4580.60883482]
車両が顧客813に到着する時刻は、[4585.70785433]
車両が顧客814に到着する時刻は、[4590.80687385]
車両が顧客815に到着する時刻は、[4596.80687385]
車両が顧客816に到着する時刻は、[4599.04294182]
車両が顧客817に到着する時刻は、[4600.04294182]
車両が顧客818に到着する時刻は、[4606.75114576]
車両が顧客819に到着する時刻は、[4612.83390829]
車両が顧客820に到着する時刻は、[4619.9049761]
車両が顧客821に到着する時刻は、[4628.39025747]
車両が顧客822に到着する時刻は、[4634.22120937]
車両が顧客823に到着する時刻は、[4641.43231192]
車両が顧客824に到着する時刻は

車両が顧客1240に到着する時刻は、[6999.18645589]
車両が顧客1241に到着する時刻は、[7006.39755844]
車両が顧客1242に到着する時刻は、[7012.72211376]
車両が顧客1243に到着する時刻は、[7019.04666908]
車両が顧客1244に到着する時刻は、[7024.43183389]
車両が顧客1245に到着する時刻は、[7028.67447457]
車両が顧客1246に到着する時刻は、[7033.14661053]
車両が顧客1247に到着する時刻は、[7040.21767834]
車両が顧客1248に到着する時刻は、[7048.02792801]
車両が顧客1249に到着する時刻は、[7054.43105225]
車両が顧客1250に到着する時刻は、[7062.04682536]
車両が顧客1251に到着する時刻は、[7065.04682536]
車両が顧客1252に到着する時刻は、[7067.28289334]
車両が顧客1253に到着する時刻は、[7072.38191285]
車両が顧客1254に到着する時刻は、[7079.45298066]
車両が顧客1255に到着する時刻は、[7082.61525832]
車両が顧客1256に到着する時刻は、[7091.21758359]
車両が顧客1257に到着する時刻は、[7094.04601071]
車両が顧客1258に到着する時刻は、[7099.04601071]
車両が顧客1259に到着する時刻は、[7104.43117552]
車両が顧客1260に到着する時刻は、[7105.84538908]
車両が顧客1261に到着する時刻は、[7111.23055389]
車両が顧客1262に到着する時刻は、[7114.39283155]
車両が顧客1263に到着する時刻は、[7119.39283155]
車両が顧客1264に到着する時刻は、[7124.39283155]
車両が顧客1265に到着する時刻は、[7128.51593718]
車両が顧客1266に到着する時刻は、[7130.51593718]
車両が顧客1267に到着する時刻は、[7135.90110198]
車両が顧客1268に到着する時刻は、[7139.06337964]
車両が顧客1269に到着する

車両が顧客1484に到着する時刻は、[8348.57905754]
車両が顧客1485に到着する時刻は、[8355.65012535]
車両が顧客1486に到着する時刻は、[8357.88619333]
車両が顧客1487に到着する時刻は、[8364.28931757]
車両が顧客1488に到着する時刻は、[8373.28931757]
車両が顧客1489に到着する時刻は、[8380.28931757]
車両が顧客1490に到着する時刻は、[8387.50042012]
車両が顧客1491に到着する時刻は、[8392.50042012]
車両が顧客1492に到着する時刻は、[8397.50042012]
車両が顧客1493に到着する時刻は、[8405.31066979]
車両が顧客1494に到着する時刻は、[8407.54673777]
車両が顧客1495に到着する時刻は、[8414.61780558]
車両が顧客1496に到着する時刻は、[8420.70056811]
車両が顧客1497に到着する時刻は、[8426.78333064]
車両が顧客1498に到着する時刻は、[8435.83871578]
車両が顧客1499に到着する時刻は、[8445.68757358]
車両が顧客1500に到着する時刻は、[8459.14119763]
車両が顧客1501に到着する時刻は、[8467.20345538]
車両が顧客1502に到着する時刻は、[8471.326561]
車両が顧客1503に到着する時刻は、[8478.53766356]
車両が顧客1504に到着する時刻は、[8482.66076918]
車両が顧客1505に到着する時刻は、[8493.84110907]
車両が顧客1506に到着する時刻は、[8503.89098469]
車両が顧客1507に到着する時刻は、[8505.30519825]
車両が顧客1508に到着する時刻は、[8515.50323728]
車両が顧客1509に到着する時刻は、[8521.58599981]
車両が顧客1510に到着する時刻は、[8526.58599981]
車両が顧客1511に到着する時刻は、[8528.82206779]
車両が顧客1512に到着する時刻は、[8531.98434545]
車両が顧客1513に到着する時刻

...

車両が顧客1429に到着する時刻は、[15694.8401848]
車両が顧客1430に到着する時刻は、[15705.28049131]
車両が顧客1431に到着する時刻は、[15708.88604259]
車両が顧客1432に到着する時刻は、[15709.88604259]
車両が顧客1433に到着する時刻は、[15713.49159386]
車両が顧客1434に到着する時刻は、[15715.72766184]
車両が顧客1435に到着する時刻は、[15719.33321312]
車両が顧客1436に到着する時刻は、[15723.45631874]
車両が顧客1437に到着する時刻は、[15731.70252999]
車両が顧客1438に到着する時刻は、[15733.70252999]
車両が顧客1439に到着する時刻は、[15744.51918382]
車両が顧客1440に到着する時刻は、[15754.0060168]
車両が顧客1441に到着する時刻は、[15756.0060168]
車両が顧客1442に到着する時刻は、[15761.0060168]
車両が顧客1443に到着する時刻は、[15767.40914104]
車両が顧客1444に到着する時刻は、[15768.40914104]
車両が顧客1445に到着する時刻は、[15772.53224666]
車両が顧客1446に到着する時刻は、[15777.53224666]
車両が顧客1447に到着する時刻は、[15790.33849514]
車両が顧客1448に到着する時刻は、[15803.14474361]
車両が顧客1449に到着する時刻は、[15813.19461923]
車両が顧客1450に到着する時刻は、[15815.43068721]
車両が顧客1451に到着する時刻は、[15824.65023167]
車両が顧客1452に到着する時刻は、[15827.47865879]
車両が顧客1453に到着する時刻は、[15832.47865879]
車両が顧客1454に到着する時刻は、[15838.13551304]
車両が顧客1455に到着する時刻は、[15846.19777079]
車両が顧客1456に到着する時刻は、[15855.68460377]
車両が顧客1457に到着する時刻は、[15863

車両が顧客1749に到着する時刻は、[17504.45793039]
車両が顧客1750に到着する時刻は、[17508.45793039]
車両が顧客1751に到着する時刻は、[17516.52018813]
車両が顧客1752に到着する時刻は、[17524.33043781]
車両が顧客1753に到着する時刻は、[17534.17929561]
車両が顧客1754に到着する時刻は、[17543.12356752]
車両が顧客1755に到着する時刻は、[17552.06783943]
車両が顧客1756に到着する時刻は、[17561.28738389]
車両が顧客1757に到着する時刻は、[17571.33725951]
車両が顧客1758に到着する時刻は、[17576.43627902]
車両が顧客1759に到着する時刻は、[17583.71638891]
車両が顧客1760に到着する時刻は、[17589.79915144]
車両が顧客1761に到着する時刻は、[17600.23945795]
車両が顧客1762に到着する時刻は、[17602.47552593]
車両が顧客1763に到着する時刻は、[17607.47552593]
車両が顧客1764に到着する時刻は、[17615.53778368]
車両が顧客1765に到着する時刻は、[17620.00991963]
車両が顧客1766に到着する時刻は、[17630.45022614]
車両が顧客1767に到着する時刻は、[17635.45022614]
車両が顧客1768に到着する時刻は、[17640.45022614]
車両が顧客1769に到着する時刻は、[17644.45022614]
車両が顧客1770に到着する時刻は、[17646.68629412]
車両が顧客1771に到着する時刻は、[17654.30206723]
車両が顧客1772に到着する時刻は、[17660.13301912]
車両が顧客1773に到着する時刻は、[17661.54723268]
車両が顧客1774に到着する時刻は、[17672.94898693]
車両が顧客1775に到着する時刻は、[17678.04800645]
車両が顧客1776に到着する時刻は、[17685.66377955]
車両が顧客1777に到着する時刻は、[1

車両が顧客2071に到着する時刻は、[19407.5415643]
車両が顧客2072に到着する時刻は、[19412.92672911]
車両が顧客2073に到着する時刻は、[19419.99779692]
車両が顧客2074に到着する時刻は、[19428.06005467]
車両が顧客2075に到着する時刻は、[19436.60405842]
車両が顧客2076に到着する時刻は、[19447.42071224]
車両が顧客2077に到着する時刻は、[19456.36498415]
車両が顧客2078に到着する時刻は、[19464.36498415]
車両が顧客2079に到着する時刻は、[19473.36498415]
車両が顧客2080に到着する時刻は、[19478.46400367]
車両が顧客2081に到着する時刻は、[19489.28065749]
車両が顧客2082に到着する時刻は、[19495.60521281]
車両が顧客2083に到着する時刻は、[19500.99037762]
車両が顧客2084に到着する時刻は、[19508.99037762]
車両が顧客2085に到着する時刻は、[19514.99037762]
車両が顧客2086に到着する時刻は、[19520.99037762]
車両が顧客2087に到着する時刻は、[19525.46251358]
車両が顧客2088に到着する時刻は、[19529.5856192]
車両が顧客2089に到着する時刻は、[19535.98874344]
車両が顧客2090に到着する時刻は、[19540.46087939]
車両が顧客2091に到着する時刻は、[19549.94771237]
車両が顧客2092に到着する時刻は、[19554.94771237]
車両が顧客2093に到着する時刻は、[19562.01878019]
車両が顧客2094に到着する時刻は、[19566.49091614]
車両が顧客2095に到着する時刻は、[19578.53251072]
車両が顧客2096に到着する時刻は、[19587.75205518]
車両が顧客2097に到着する時刻は、[19592.75205518]
車両が顧客2098に到着する時刻は、[19602.9500942]
車両が顧客2099に到着する時刻は、[1961

車両が顧客2437に到着する時刻は、[21555.2206527]
車両が顧客2438に到着する時刻は、[21561.0516046]
車両が顧客2439に到着する時刻は、[21566.0516046]
車両が顧客2440に到着する時刻は、[21569.65715587]
車両が顧客2441に到着する時刻は、[21574.75617538]
車両が顧客2442に到着する時刻は、[21582.03628527]
車両が顧客2443に到着する時刻は、[21585.64183655]
車両が顧客2444に到着する時刻は、[21588.80411421]
車両が顧客2445に到着する時刻は、[21597.05032546]
車両が顧客2446に到着する時刻は、[21600.05032546]
車両が顧客2447に到着する時刻は、[21608.59432921]
車両が顧客2448に到着する時刻は、[21613.69334872]
車両が顧客2449に到着する時刻は、[21614.69334872]
車両が顧客2450に到着する時刻は、[21621.01790404]
車両が顧客2451に到着する時刻は、[21623.25397202]
車両が顧客2452に到着する時刻は、[21632.47351647]
車両が顧客2453に到着する時刻は、[21638.79807179]
車両が顧客2454に到着する時刻は、[21644.45492604]
車両が顧客2455に到着する時刻は、[21648.927062]
車両が顧客2456に到着する時刻は、[21657.98244714]
車両が顧客2457に到着する時刻は、[21661.98244714]
車両が顧客2458に到着する時刻は、[21665.1447248]
車両が顧客2459に到着する時刻は、[21672.1447248]
車両が顧客2460に到着する時刻は、[21681.9935826]
車両が顧客2461に到着する時刻は、[21691.89307754]
車両が顧客2462に到着する時刻は、[21695.89307754]
車両が顧客2463に到着する時刻は、[21697.3072911]
車両が顧客2464に到着する時刻は、[21699.54335908]
車両が顧客2465に到着する時刻は、[21703.7859

車両が顧客2991に到着する時刻は、[24593.23350979]
車両が顧客2992に到着する時刻は、[24596.39578745]
車両が顧客2993に到着する時刻は、[24601.49480697]
車両が顧客2994に到着する時刻は、[24607.8979312]
車両が顧客2995に到着する時刻は、[24615.10903375]
車両が顧客2996に到着する時刻は、[24623.71135902]
車両が顧客2997に到着する時刻は、[24628.81037853]
車両が顧客2998に到着する時刻は、[24632.41592981]
車両が顧客2999に到着する時刻は、[24642.85623632]
車両が顧客3000に到着する時刻は、[24652.85623632]
車両が顧客3001に到着する時刻は、[24657.09887701]
車両が顧客3002に到着する時刻は、[24665.64288075]
車両が顧客3003に到着する時刻は、[24677.68447533]
車両が顧客3004に到着する時刻は、[24688.31462114]
車両が顧客3005に到着する時刻は、[24696.37687889]
車両が顧客3006に到着する時刻は、[24704.92088264]
車両が顧客3007に到着する時刻は、[24713.97626777]
車両が顧客3008に到着する時刻は、[24722.52027152]
車両が顧客3009に到着する時刻は、[24724.52027152]
車両が顧客3010に到着する時刻は、[24726.7563395]
車両が顧客3011に到着する時刻は、[24731.7563395]
車両が顧客3012に到着する時刻は、[24737.41319375]
車両が顧客3013に到着する時刻は、[24741.01874502]
車両が顧客3014に到着する時刻は、[24750.23828948]
車両が顧客3015に到着する時刻は、[24753.84384076]
車両が顧客3016に到着する時刻は、[24762.78811267]
車両が顧客3017に到着する時刻は、[24771.73238458]
車両が顧客3018に到着する時刻は、[24779.01249446]
車両が顧客3019に到着する時刻は、[2478

In [ ]:
for v in former_x.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for v in latter_x.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for constr in former_x.getConstrs()+latter_x.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [17]:
# 初期解の保存
basis_like = np.array([constr.Pi for constr in former_x.getConstrs()+latter_x.getConstrs()])
y3 = np.zeros((Ax.shape[0]-basis_like.shape[0],))
basis_like = np.append(basis_like, y3)

In [ ]:
print(f"Ax.T.shape={Ax.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for Ax_i, Ap_i in zip(Ax.T, Ap.T):
    print(Ax_i, "\t", Ap_i)

In [20]:
# Gurobiによって最適解を求める
whole_y = solve_dual(Ax, Ap, b, c, instance_name+"whole_y", basis_like.shape[0], basis_like)

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 31609 rows, 18965 columns and 56895 nonzeros
Model fingerprint: 0x5c70e6a6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   6.980696e+07      0s
   23864   -4.3887602e+07   0.000000e+00   0.000000e+00      4s

Solved in 23864 iterations and 4.21 seconds
Optimal objective -4.388760208e+07
主問題における制約0の潜在価格は、[6317.]
主問題における制約1の潜在価格は、[6318.]
主問題における制約2の潜在価格は、[6319.]
主問題における制約3の潜在価格は、[6318.]
主問題における制約4の潜在価格は、[6317.]
主問題における制約5の潜在価格は、[6316.]
主問題における制約6の潜在価格は、[6315.]
主問題における制約7の潜在価格は、[6314.]
主問題における制約8の潜在価格は、[6313.]
主問題における制約9の潜在価格は、[6312.]
主問

主問題における制約316の潜在価格は、[6005.]
主問題における制約317の潜在価格は、[6004.]
主問題における制約318の潜在価格は、[6003.]
主問題における制約319の潜在価格は、[6002.]
主問題における制約320の潜在価格は、[6001.]
主問題における制約321の潜在価格は、[6000.]
主問題における制約322の潜在価格は、[5999.]
主問題における制約323の潜在価格は、[5998.]
主問題における制約324の潜在価格は、[5997.]
主問題における制約325の潜在価格は、[5996.]
主問題における制約326の潜在価格は、[5995.]
主問題における制約327の潜在価格は、[5994.]
主問題における制約328の潜在価格は、[5993.]
主問題における制約329の潜在価格は、[5992.]
主問題における制約330の潜在価格は、[5991.]
主問題における制約331の潜在価格は、[5990.]
主問題における制約332の潜在価格は、[5989.]
主問題における制約333の潜在価格は、[5988.]
主問題における制約334の潜在価格は、[5987.]
主問題における制約335の潜在価格は、[5986.]
主問題における制約336の潜在価格は、[5985.]
主問題における制約337の潜在価格は、[5984.]
主問題における制約338の潜在価格は、[5983.]
主問題における制約339の潜在価格は、[5982.]
主問題における制約340の潜在価格は、[5981.]
主問題における制約341の潜在価格は、[5980.]
主問題における制約342の潜在価格は、[5979.]
主問題における制約343の潜在価格は、[5978.]
主問題における制約344の潜在価格は、[5977.]
主問題における制約345の潜在価格は、[5976.]
主問題における制約346の潜在価格は、[5975.]
主問題における制約347の潜在価格は、[5974.]
主問題における制約348の潜在価格は、[5973.]
主問題における制約349の潜在価格は、[5972.]
主問題における制約350の潜在価格は、[5971.]
主問題における制約351の潜在価格は、[5970.]
主問題における制約352の潜在価格は、[5969.]
主

主問題における制約623の潜在価格は、[5698.]
主問題における制約624の潜在価格は、[5697.]
主問題における制約625の潜在価格は、[5696.]
主問題における制約626の潜在価格は、[5695.]
主問題における制約627の潜在価格は、[5694.]
主問題における制約628の潜在価格は、[5693.]
主問題における制約629の潜在価格は、[5692.]
主問題における制約630の潜在価格は、[5691.]
主問題における制約631の潜在価格は、[5690.]
主問題における制約632の潜在価格は、[5689.]
主問題における制約633の潜在価格は、[5688.]
主問題における制約634の潜在価格は、[5687.]
主問題における制約635の潜在価格は、[5686.]
主問題における制約636の潜在価格は、[5685.]
主問題における制約637の潜在価格は、[5684.]
主問題における制約638の潜在価格は、[5683.]
主問題における制約639の潜在価格は、[5682.]
主問題における制約640の潜在価格は、[5681.]
主問題における制約641の潜在価格は、[5680.]
主問題における制約642の潜在価格は、[5679.]
主問題における制約643の潜在価格は、[5678.]
主問題における制約644の潜在価格は、[5677.]
主問題における制約645の潜在価格は、[5676.]
主問題における制約646の潜在価格は、[5675.]
主問題における制約647の潜在価格は、[5674.]
主問題における制約648の潜在価格は、[5673.]
主問題における制約649の潜在価格は、[5672.]
主問題における制約650の潜在価格は、[5671.]
主問題における制約651の潜在価格は、[5670.]
主問題における制約652の潜在価格は、[5669.]
主問題における制約653の潜在価格は、[5668.]
主問題における制約654の潜在価格は、[5667.]
主問題における制約655の潜在価格は、[5666.]
主問題における制約656の潜在価格は、[5665.]
主問題における制約657の潜在価格は、[5664.]
主問題における制約658の潜在価格は、[5663.]
主問題における制約659の潜在価格は、[5662.]
主

主問題における制約1069の潜在価格は、[5252.]
主問題における制約1070の潜在価格は、[5251.]
主問題における制約1071の潜在価格は、[5250.]
主問題における制約1072の潜在価格は、[5249.]
主問題における制約1073の潜在価格は、[5248.]
主問題における制約1074の潜在価格は、[5247.]
主問題における制約1075の潜在価格は、[5246.]
主問題における制約1076の潜在価格は、[5245.]
主問題における制約1077の潜在価格は、[5244.]
主問題における制約1078の潜在価格は、[5243.]
主問題における制約1079の潜在価格は、[5242.]
主問題における制約1080の潜在価格は、[5241.]
主問題における制約1081の潜在価格は、[5240.]
主問題における制約1082の潜在価格は、[5239.]
主問題における制約1083の潜在価格は、[5238.]
主問題における制約1084の潜在価格は、[5237.]
主問題における制約1085の潜在価格は、[5236.]
主問題における制約1086の潜在価格は、[5235.]
主問題における制約1087の潜在価格は、[5234.]
主問題における制約1088の潜在価格は、[5233.]
主問題における制約1089の潜在価格は、[5232.]
主問題における制約1090の潜在価格は、[5231.]
主問題における制約1091の潜在価格は、[5230.]
主問題における制約1092の潜在価格は、[5229.]
主問題における制約1093の潜在価格は、[5228.]
主問題における制約1094の潜在価格は、[5227.]
主問題における制約1095の潜在価格は、[5226.]
主問題における制約1096の潜在価格は、[5225.]
主問題における制約1097の潜在価格は、[5224.]
主問題における制約1098の潜在価格は、[5223.]
主問題における制約1099の潜在価格は、[5222.]
主問題における制約1100の潜在価格は、[5221.]
主問題における制約1101の潜在価格は、[5220.]
主問題における制約1102の潜在価格は、[5219.]
主問題における制約1103の潜在価格は、[5218.]
主問題における制約1104の潜在価格は、

主問題における制約1484の潜在価格は、[4837.]
主問題における制約1485の潜在価格は、[4836.]
主問題における制約1486の潜在価格は、[4835.]
主問題における制約1487の潜在価格は、[4834.]
主問題における制約1488の潜在価格は、[4833.]
主問題における制約1489の潜在価格は、[4832.]
主問題における制約1490の潜在価格は、[4831.]
主問題における制約1491の潜在価格は、[4830.]
主問題における制約1492の潜在価格は、[4829.]
主問題における制約1493の潜在価格は、[4828.]
主問題における制約1494の潜在価格は、[4827.]
主問題における制約1495の潜在価格は、[4826.]
主問題における制約1496の潜在価格は、[4825.]
主問題における制約1497の潜在価格は、[4824.]
主問題における制約1498の潜在価格は、[4823.]
主問題における制約1499の潜在価格は、[4822.]
主問題における制約1500の潜在価格は、[4821.]
主問題における制約1501の潜在価格は、[4820.]
主問題における制約1502の潜在価格は、[4819.]
主問題における制約1503の潜在価格は、[4818.]
主問題における制約1504の潜在価格は、[4817.]
主問題における制約1505の潜在価格は、[4816.]
主問題における制約1506の潜在価格は、[4815.]
主問題における制約1507の潜在価格は、[4814.]
主問題における制約1508の潜在価格は、[4813.]
主問題における制約1509の潜在価格は、[4812.]
主問題における制約1510の潜在価格は、[4811.]
主問題における制約1511の潜在価格は、[4810.]
主問題における制約1512の潜在価格は、[4809.]
主問題における制約1513の潜在価格は、[4808.]
主問題における制約1514の潜在価格は、[4807.]
主問題における制約1515の潜在価格は、[4806.]
主問題における制約1516の潜在価格は、[4805.]
主問題における制約1517の潜在価格は、[4804.]
主問題における制約1518の潜在価格は、[4803.]
主問題における制約1519の潜在価格は、

...

主問題における制約17090の潜在価格は、[1.]
主問題における制約17091の潜在価格は、[0.]
主問題における制約17092の潜在価格は、[1.]
主問題における制約17093の潜在価格は、[0.]
主問題における制約17094の潜在価格は、[1.]
主問題における制約17095の潜在価格は、[0.]
主問題における制約17096の潜在価格は、[1.]
主問題における制約17097の潜在価格は、[0.]
主問題における制約17098の潜在価格は、[1.]
主問題における制約17099の潜在価格は、[0.]
主問題における制約17100の潜在価格は、[1.]
主問題における制約17101の潜在価格は、[0.]
主問題における制約17102の潜在価格は、[1.]
主問題における制約17103の潜在価格は、[0.]
主問題における制約17104の潜在価格は、[1.]
主問題における制約17105の潜在価格は、[0.]
主問題における制約17106の潜在価格は、[1.]
主問題における制約17107の潜在価格は、[0.]
主問題における制約17108の潜在価格は、[1.]
主問題における制約17109の潜在価格は、[0.]
主問題における制約17110の潜在価格は、[1.]
主問題における制約17111の潜在価格は、[0.]
主問題における制約17112の潜在価格は、[1.]
主問題における制約17113の潜在価格は、[0.]
主問題における制約17114の潜在価格は、[1.]
主問題における制約17115の潜在価格は、[0.]
主問題における制約17116の潜在価格は、[1.]
主問題における制約17117の潜在価格は、[0.]
主問題における制約17118の潜在価格は、[1.]
主問題における制約17119の潜在価格は、[0.]
主問題における制約17120の潜在価格は、[1.]
主問題における制約17121の潜在価格は、[0.]
主問題における制約17122の潜在価格は、[1.]
主問題における制約17123の潜在価格は、[0.]
主問題における制約17124の潜在価格は、[1.]
主問題における制約17125の潜在価格は、[0.]
主問題における制約17126の潜在価格は、[1.]
主問題における制約17127の潜在価格は、[0.]
主問題における制約171

主問題における制約17408の潜在価格は、[1.]
主問題における制約17409の潜在価格は、[0.]
主問題における制約17410の潜在価格は、[1.]
主問題における制約17411の潜在価格は、[0.]
主問題における制約17412の潜在価格は、[1.]
主問題における制約17413の潜在価格は、[0.]
主問題における制約17414の潜在価格は、[1.]
主問題における制約17415の潜在価格は、[0.]
主問題における制約17416の潜在価格は、[1.]
主問題における制約17417の潜在価格は、[0.]
主問題における制約17418の潜在価格は、[1.]
主問題における制約17419の潜在価格は、[0.]
主問題における制約17420の潜在価格は、[1.]
主問題における制約17421の潜在価格は、[0.]
主問題における制約17422の潜在価格は、[1.]
主問題における制約17423の潜在価格は、[0.]
主問題における制約17424の潜在価格は、[1.]
主問題における制約17425の潜在価格は、[0.]
主問題における制約17426の潜在価格は、[1.]
主問題における制約17427の潜在価格は、[0.]
主問題における制約17428の潜在価格は、[1.]
主問題における制約17429の潜在価格は、[0.]
主問題における制約17430の潜在価格は、[1.]
主問題における制約17431の潜在価格は、[0.]
主問題における制約17432の潜在価格は、[1.]
主問題における制約17433の潜在価格は、[0.]
主問題における制約17434の潜在価格は、[1.]
主問題における制約17435の潜在価格は、[0.]
主問題における制約17436の潜在価格は、[1.]
主問題における制約17437の潜在価格は、[0.]
主問題における制約17438の潜在価格は、[1.]
主問題における制約17439の潜在価格は、[0.]
主問題における制約17440の潜在価格は、[1.]
主問題における制約17441の潜在価格は、[0.]
主問題における制約17442の潜在価格は、[1.]
主問題における制約17443の潜在価格は、[0.]
主問題における制約17444の潜在価格は、[1.]
主問題における制約17445の潜在価格は、[0.]
主問題における制約174

主問題における制約17883の潜在価格は、[0.]
主問題における制約17884の潜在価格は、[1.]
主問題における制約17885の潜在価格は、[0.]
主問題における制約17886の潜在価格は、[1.]
主問題における制約17887の潜在価格は、[0.]
主問題における制約17888の潜在価格は、[1.]
主問題における制約17889の潜在価格は、[0.]
主問題における制約17890の潜在価格は、[1.]
主問題における制約17891の潜在価格は、[0.]
主問題における制約17892の潜在価格は、[1.]
主問題における制約17893の潜在価格は、[0.]
主問題における制約17894の潜在価格は、[1.]
主問題における制約17895の潜在価格は、[0.]
主問題における制約17896の潜在価格は、[1.]
主問題における制約17897の潜在価格は、[0.]
主問題における制約17898の潜在価格は、[1.]
主問題における制約17899の潜在価格は、[0.]
主問題における制約17900の潜在価格は、[1.]
主問題における制約17901の潜在価格は、[0.]
主問題における制約17902の潜在価格は、[1.]
主問題における制約17903の潜在価格は、[0.]
主問題における制約17904の潜在価格は、[1.]
主問題における制約17905の潜在価格は、[0.]
主問題における制約17906の潜在価格は、[1.]
主問題における制約17907の潜在価格は、[0.]
主問題における制約17908の潜在価格は、[1.]
主問題における制約17909の潜在価格は、[0.]
主問題における制約17910の潜在価格は、[1.]
主問題における制約17911の潜在価格は、[0.]
主問題における制約17912の潜在価格は、[1.]
主問題における制約17913の潜在価格は、[0.]
主問題における制約17914の潜在価格は、[1.]
主問題における制約17915の潜在価格は、[0.]
主問題における制約17916の潜在価格は、[1.]
主問題における制約17917の潜在価格は、[0.]
主問題における制約17918の潜在価格は、[1.]
主問題における制約17919の潜在価格は、[0.]
主問題における制約17920の潜在価格は、[1.]
主問題における制約179

主問題における制約18258の潜在価格は、[1.]
主問題における制約18259の潜在価格は、[0.]
主問題における制約18260の潜在価格は、[1.]
主問題における制約18261の潜在価格は、[0.]
主問題における制約18262の潜在価格は、[1.]
主問題における制約18263の潜在価格は、[0.]
主問題における制約18264の潜在価格は、[1.]
主問題における制約18265の潜在価格は、[0.]
主問題における制約18266の潜在価格は、[1.]
主問題における制約18267の潜在価格は、[0.]
主問題における制約18268の潜在価格は、[1.]
主問題における制約18269の潜在価格は、[0.]
主問題における制約18270の潜在価格は、[1.]
主問題における制約18271の潜在価格は、[0.]
主問題における制約18272の潜在価格は、[1.]
主問題における制約18273の潜在価格は、[0.]
主問題における制約18274の潜在価格は、[1.]
主問題における制約18275の潜在価格は、[0.]
主問題における制約18276の潜在価格は、[1.]
主問題における制約18277の潜在価格は、[0.]
主問題における制約18278の潜在価格は、[1.]
主問題における制約18279の潜在価格は、[0.]
主問題における制約18280の潜在価格は、[1.]
主問題における制約18281の潜在価格は、[0.]
主問題における制約18282の潜在価格は、[1.]
主問題における制約18283の潜在価格は、[0.]
主問題における制約18284の潜在価格は、[1.]
主問題における制約18285の潜在価格は、[0.]
主問題における制約18286の潜在価格は、[1.]
主問題における制約18287の潜在価格は、[0.]
主問題における制約18288の潜在価格は、[1.]
主問題における制約18289の潜在価格は、[0.]
主問題における制約18290の潜在価格は、[1.]
主問題における制約18291の潜在価格は、[0.]
主問題における制約18292の潜在価格は、[1.]
主問題における制約18293の潜在価格は、[0.]
主問題における制約18294の潜在価格は、[1.]
主問題における制約18295の潜在価格は、[0.]
主問題における制約182

主問題における制約18635の潜在価格は、[0.]
主問題における制約18636の潜在価格は、[1.]
主問題における制約18637の潜在価格は、[0.]
主問題における制約18638の潜在価格は、[1.]
主問題における制約18639の潜在価格は、[0.]
主問題における制約18640の潜在価格は、[1.]
主問題における制約18641の潜在価格は、[0.]
主問題における制約18642の潜在価格は、[1.]
主問題における制約18643の潜在価格は、[0.]
主問題における制約18644の潜在価格は、[1.]
主問題における制約18645の潜在価格は、[0.]
主問題における制約18646の潜在価格は、[1.]
主問題における制約18647の潜在価格は、[0.]
主問題における制約18648の潜在価格は、[1.]
主問題における制約18649の潜在価格は、[0.]
主問題における制約18650の潜在価格は、[1.]
主問題における制約18651の潜在価格は、[0.]
主問題における制約18652の潜在価格は、[1.]
主問題における制約18653の潜在価格は、[0.]
主問題における制約18654の潜在価格は、[1.]
主問題における制約18655の潜在価格は、[0.]
主問題における制約18656の潜在価格は、[1.]
主問題における制約18657の潜在価格は、[0.]
主問題における制約18658の潜在価格は、[1.]
主問題における制約18659の潜在価格は、[0.]
主問題における制約18660の潜在価格は、[1.]
主問題における制約18661の潜在価格は、[0.]
主問題における制約18662の潜在価格は、[1.]
主問題における制約18663の潜在価格は、[0.]
主問題における制約18664の潜在価格は、[1.]
主問題における制約18665の潜在価格は、[0.]
主問題における制約18666の潜在価格は、[1.]
主問題における制約18667の潜在価格は、[0.]
主問題における制約18668の潜在価格は、[1.]
主問題における制約18669の潜在価格は、[0.]
主問題における制約18670の潜在価格は、[1.]
主問題における制約18671の潜在価格は、[0.]
主問題における制約18672の潜在価格は、[1.]
主問題における制約186

In [ ]:
# 最適解
for var in whole_y.getVars():
    print(var.varName, var.X)

### 4. 全体のPrimalの最適解を得る

In [ ]:
for i, constr in enumerate(whole_y.getConstrs()):
    print(constr.Pi)

In [ ]:
for var in whole_x.getVars():
    print(var.varName, var.X)